In [ ]:
import torch
from torch.autograd import Variable
import numpy as np
import pandas as pd
import torch.nn.functional as F
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
datapath = "/content/drive/My Drive/DevC/datasets/VNTQcorpus.txt"
phone_path = "/content/drive/My Drive/DevC/datasets/phone.csv"
earphone_path = "/content/drive/My Drive/DevC/datasets/earphone.csv"
laptop_path = "/content/drive/My Drive/DevC/datasets/laptop.csv"
with open(datapath, "r") as file:
  data = file.read().splitlines()
phone_data = pd.read_csv(phone_path)
earphone_data = pd.read_csv(earphone_path)
laptop_data = pd.read_csv(laptop_path) 
print(data[:10])

['Tìm :.', 'Tên Sách.', 'Tên Tác Giả.', 'Tìm.', 'theo tên Tác Giả.', 'Tìm theo Thể Loại.', 'Lịch sử.', 'Cổ tích.', 'Tuổi Học Trò.', 'Cổ Văn Việt Nam.']


In [ ]:
phone_data = phone_data[["content"]]
phone_data.head(5)

,content
0,Giao hàng khá nhanh.
1,Tầm chưa đầy 2 ngày là giao hàng.
2,Thấy mn nói không có lót giấy nhưng của mình t...
3,Hộp còn nguyên seal.
4,Pin lúc khởi động là 55% xài tầm 30p còn 50%.


In [ ]:
earphone_data = earphone_data[["content"]]
earphone_data.head(5)

,content
0,"Mua hàng chốt đơn nhận tiền rất nhanh, không k..."
1,nhưng nhận hàng thì 5 ngày rồi chưa kích foajt...
2,Rất không hài lòng với hậu mãi như Tiki.
3,"Lúc mua cái này mình phân vân lắm, tại tới tận..."
4,"chất lượng tai nghe rất tốt, nhưng:\nđồ công n..."


In [ ]:
laptop_data = laptop_data[["content"]]
laptop_data.head(5)

,content
0,sao cách đây 3 hôm Tiki cũng chạy khuyến mãi n...
1,mã đơn hàng mình : 975928400
2,"Mình mua sản phẩm theo hình thức trả góp, nhưn..."
3,"1 tháng sau ngày mua, BIDV rút nợ tự động tron..."
4,Số dư trong tài khoản không đủ nên rút nợ khôn...


In [ ]:
tiki_data = pd.concat([phone_data, laptop_data, earphone_data])

In [ ]:
tiki_data = tiki_data.sample(frac = 1).reset_index(drop = True)
tiki_data.head(10)

,content
0,Màu xanh dương này nhìn siêu xinh.
1,Mình mới nhận được hàng sáng nay nhưng sử dụng...
2,"Tai nghe nghe rất phê, đối với mình đây là một..."
3,trên tiki ghi là sản suất ở trung Quốc nhưng t...
4,lúc thì ko bắt đc tai trái.
5,"giao hàng rất nhanh, đóng gói kĩ lưỡng, chất l..."
6,cảm ứng ok .pause _play -tăng giảm âm lượng ok
7,"Hàng chính hãng, mới sx năm 2019 luôn."
8,Âm thanh hơi bé.
9,Mua cái đầu tiên xài được 1 tháng thì bị hỏng ...


In [ ]:
tiki_data = tiki_data.content.to_list()

In [ ]:
ev_path = "/content/drive/My Drive/DevC/process materials/Englishwords.xlsx"
sf_path = "/content/drive/My Drive/DevC/process materials/Shortform.xlsx"

In [ ]:
englishwords = pd.read_excel(ev_path, index_col = "English")
print(englishwords.head(10))

shortform = pd.read_excel(sf_path, index_col= "Short")
print(shortform.head(10))

            Vietnamese
English               
access        truy cập
adapter        cục sạc
ah                   à
ak                   à
app           ứng dụng
application   ứng dụng
avoid            tránh
book              sách
bye           tạm biệt
camera         máy ảnh
               Long
Short              
ace      anh chị em
ae           anh em
bb          bạn bè 
bcuoi     buồn cười
bg          bây giờ
bh          bây giờ
bn        bao nhiêu
bt      bình thường
bthg    bình thường
chs    chả hiểu sao


In [ ]:
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)
def preprocess(text):
  #bỏ tag html và emoji
  text = re.sub('<[^>]*>', '', text)
  text = deEmojify(text)

  #thay chữ cái viết hoa thành viết thường
  text = text.lower()

  #xóa dấu ngắt câu, xóa link và các chữ có chứa chữ số
  clean_text = []
  punc_list = '.,;:?!\|/&@`~()-_@#$%^*'
  for w in (text.split()):
    if "http" in w:
      continue
    clean_text.append(w)
  text = ' '.join(clean_text)
  for punc in punc_list:
    text = text.replace(punc, ' ')

  #xóa bỏ các chữ cái lặp liên tiếp nhau (đỉnhhhhhhhhhh, vipppppppppppppppp)
  length = len(text)
  char = 0
  while char <length-1:
    if text[char] == text[char+1]:
      text = text[:char]+text[char+1:]
      #print(text)
      length-=1
      continue
    char+=1  
  numbers = ["không", "một", "hai", "ba", "bốn", "năm", "sáu", "bảy", "tám", "chín"]
  #chuyển đổi các từ tiếng anh và viết tắt thông dụng sang tiếng Việt chuẩn:
  text_split = text.split()
  for i, w in enumerate(text_split):
    if w in englishwords.index:
      text_split[i] = str(englishwords.loc[w, "Vietnamese"])
    if w in shortform.index:
      text_split[i] = str(shortform.loc[w, "Long"])
    if w.isdigit():
      text_split[i] = ' '.join([numbers[int(c)] for c in w]) 
  text = ' '.join(text_split)

  #loại bỏ tất cả các kí tự đặc biệt còn lại
  digits_and_characters = 'aăâbcdđeêfghijklmnoôơpqrstuưvxywzáàảãạắằẳẵặấầẩẫậéèẻẽẹếềểễệíìỉĩịóòỏõọốồổỗộớờởỡợúùủũụứừửữựýỳỷỹỵ0123456789 '
  text = ''.join([i for i in text if i in digits_and_characters])
  return text

In [ ]:
print(preprocess("Anh đẹp trai ngầu vcl, koooooooo thích tí nào ncl ko thích \U0001f602  6789 not phone <đá;jasdf> https//:facebook.com/damm nỏm1e" ))

anh đẹp trai ngầu vcl không thích tí nào nói chung là không thích sáu bảy tám chín không điện thoại nỏm1e


In [ ]:
for index, text in enumerate(tiki_data):
  #print(index, text)
  tiki_data[index] = preprocess(text)

for index, text in enumerate(data):
  data[index] = preprocess(text)

In [ ]:
print(data[:10])

['tìm', 'tên sách', 'tên tác giả', 'tìm', 'theo tên tác giả', 'tìm theo thể loại', 'lịch sử', 'cổ tích', 'tuổi học trò', 'cổ văn việt nam']


In [ ]:
print(tiki_data[:10])

['màu xanh dương này nhìn siêu xinh', 'mình mới nhận được hàng sáng nay nhưng sử dụng hay bị mất kết nối ngắt kết nối với tắt đột ngột lúc nghe được lúc không âm thanh không cân bằng với lại nhấp không nhạy mong tiki sớm sửa chữa bảo hành lại dùm mình', 'tai nghe nghe rất phê đối với mình đây là một tai nghe tuyệt vời so với giá rẻ nhưng chất lượng cao bas gõ phê', 'trên tiki ghi là sản suất ở trung quốc nhưng trên hộp lại ghi nhập từ thái lan nên cũng rất yên tâm', 'lúc thì không bắt được tai trái', 'giao hàng rất nhanh đóng gói kĩ lưỡng chất lượng xài ngje rất êm không có lỗi ở sản phẩm với giá thành như vậy thì trên cả tuyệt vời', 'cảm ứng ổn pause play tăng giảm âm lượng ổn', 'hàng chính hãng mới sx năm hai không một chín luôn', 'âm thanh hơi bé', 'mua cái đầu tiên xài được một tháng thì bị hỏng một bên không nghe được mang đi bảo hành thì delay hơn cả tháng cứ nói là đang bảo hành mà không hẹn ngày xong']


In [ ]:
def splitCorpus(corpus):
    tokens = [sentence.split() for sentence in corpus]
    return tokens

general_corpus = splitCorpus(data)
tiki_corpus = splitCorpus(tiki_data)

In [ ]:
print(general_corpus[:10])

[['tìm'], ['tên', 'sách'], ['tên', 'tác', 'giả'], ['tìm'], ['theo', 'tên', 'tác', 'giả'], ['tìm', 'theo', 'thể', 'loại'], ['lịch', 'sử'], ['cổ', 'tích'], ['tuổi', 'học', 'trò'], ['cổ', 'văn', 'việt', 'nam']]


In [ ]:
print(tiki_corpus[:10])

[['màu', 'xanh', 'dương', 'này', 'nhìn', 'siêu', 'xinh'], ['mình', 'mới', 'nhận', 'được', 'hàng', 'sáng', 'nay', 'nhưng', 'sử', 'dụng', 'hay', 'bị', 'mất', 'kết', 'nối', 'ngắt', 'kết', 'nối', 'với', 'tắt', 'đột', 'ngột', 'lúc', 'nghe', 'được', 'lúc', 'không', 'âm', 'thanh', 'không', 'cân', 'bằng', 'với', 'lại', 'nhấp', 'không', 'nhạy', 'mong', 'tiki', 'sớm', 'sửa', 'chữa', 'bảo', 'hành', 'lại', 'dùm', 'mình'], ['tai', 'nghe', 'nghe', 'rất', 'phê', 'đối', 'với', 'mình', 'đây', 'là', 'một', 'tai', 'nghe', 'tuyệt', 'vời', 'so', 'với', 'giá', 'rẻ', 'nhưng', 'chất', 'lượng', 'cao', 'bas', 'gõ', 'phê'], ['trên', 'tiki', 'ghi', 'là', 'sản', 'suất', 'ở', 'trung', 'quốc', 'nhưng', 'trên', 'hộp', 'lại', 'ghi', 'nhập', 'từ', 'thái', 'lan', 'nên', 'cũng', 'rất', 'yên', 'tâm'], ['lúc', 'thì', 'không', 'bắt', 'được', 'tai', 'trái'], ['giao', 'hàng', 'rất', 'nhanh', 'đóng', 'gói', 'kĩ', 'lưỡng', 'chất', 'lượng', 'xài', 'ngje', 'rất', 'êm', 'không', 'có', 'lỗi', 'ở', 'sản', 'phẩm', 'với', 'giá', 'thàn

In [ ]:
print(len(general_corpus))
print(len(tiki_corpus))

1765377
299414


In [ ]:
from gensim.models.phrases import Phrases, Phraser

In [ ]:
phrases = Phrases(general_corpus, min_count=50, progress_per=10000)

In [ ]:
phrases.save("/content/drive/My Drive/DevC/saves/general_phrases.pkl")

In [ ]:
bigram = Phraser(phrases)

In [ ]:
bigram.save("/content/drive/My Drive/DevC/saves/bigram.pkl")

In [ ]:
general_sentences = bigram[general_corpus]
tiki_sentences = bigram[tiki_corpus]

In [ ]:
a1 = [len(sentence) for sentence in general_corpus[:10]]
a2 = [len(sentence) for sentence in general_sentences[:10]] 
print(a1)
print(a2)

[1, 2, 3, 1, 4, 4, 2, 2, 3, 4]
[1, 2, 2, 1, 3, 4, 1, 2, 3, 3]


In [ ]:
from gensim.models import Word2Vec

In [ ]:
model = Word2Vec(size=200,sample = 6e-5, alpha = 0.03, min_alpha= 0.0007, negative = 20, window=2, min_count=50, workers=4, compute_loss = True)

In [ ]:
from time import time
t = time()

model.build_vocab(sentences= general_sentences, progress_per=10000)
print(model.wv.vocab)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

{'tìm': <gensim.models.keyedvectors.Vocab object at 0x7f9faffe3048>, 'tên': <gensim.models.keyedvectors.Vocab object at 0x7f9f986095f8>, 'sách': <gensim.models.keyedvectors.Vocab object at 0x7f9f986047f0>, 'tác_giả': <gensim.models.keyedvectors.Vocab object at 0x7f9f986045f8>, 'theo': <gensim.models.keyedvectors.Vocab object at 0x7f9f98604f60>, 'thể': <gensim.models.keyedvectors.Vocab object at 0x7f9f98604d68>, 'loại': <gensim.models.keyedvectors.Vocab object at 0x7f9f98604be0>, 'lịch_sử': <gensim.models.keyedvectors.Vocab object at 0x7f9f98604eb8>, 'cổ': <gensim.models.keyedvectors.Vocab object at 0x7f9f98604d30>, 'tích': <gensim.models.keyedvectors.Vocab object at 0x7f9f98604dd8>, 'tuổi': <gensim.models.keyedvectors.Vocab object at 0x7f9f98604550>, 'học': <gensim.models.keyedvectors.Vocab object at 0x7f9f98604278>, 'trò': <gensim.models.keyedvectors.Vocab object at 0x7f9f98604b70>, 'văn': <gensim.models.keyedvectors.Vocab object at 0x7f9f98604c50>, 'việt_nam': <gensim.models.keyedvec

In [ ]:
print(len(model.wv.vocab))
print(model.wv.vocab.keys())

11901
dict_keys(['tìm', 'tên', 'sách', 'tác_giả', 'theo', 'thể', 'loại', 'lịch_sử', 'cổ', 'tích', 'tuổi', 'học', 'trò', 'văn', 'việt_nam', 'tập', 'truyện_ngắn', 'suy_ngẫm', 'làm', 'người', 'nhân', 'vật', 'y', 'sức_khỏe', 'tình', 'cảm', 'xã_hội', 'phiêu_lưu', 'mạo_hiểm', 'hồi', 'ký', 'tuỳ', 'bút', 'điển', 'nước', 'ngoài', 'tôn', 'giáo', 'chính', 'trị', 'kịch', 'bản', 'khoa_học', 'huyền', 'bí', 'giả', 'tưởng', 'kiếm', 'hiệp', 'âm', 'dương', 'tam', 'thư', 'sinh', 'số_lần', 'đọc', 'bảy', 'tám', 'hai', 'bốn', 'sáu', 'ba', 'mục', 'lục', 'trước', 'khi', 'vào', 'truyện', 'cùng', 'một', 'chiến', 'yên', 'hùng', 'cái', 'đàn', 'chỉ', 'thần', 'công', 'điệu', 'sáo', 'mê', 'hồn', 'hải', 'nộ', 'triều', 'cỡ', 'chữ', 'không', 'thử', 'hỏi', 'tứ', 'tiên', 'và', 'võ', 'thế', 'nào', 'mà', 'vừa', 'xông', 'trong', 'giường', 'bát', 'bảo', 'đã', 'bị', 'bật', 'văng', 'ra', 'như', 'những', 'trái', 'bóng', 'nấy_đều', 'ngất_xỉu', 'mặt', 'mũi', 'tím', 'tái', 'đó', 'xét', 'của', 'di', 'nương', 'tuệ_mẫn', 'ở', 'phải',

In [ ]:
t = time()
model.train(sentences = general_sentences, total_examples=model.corpus_count, epochs=20, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to train the model: 77.36 mins


In [ ]:
model.most_similar(positive=[ "tuyệt_vời"])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('hoàn_hảo', 0.6000055074691772),
 ('đẹp', 0.5950585007667542),
 ('hấp_dẫn', 0.5603220462799072),
 ('đẹp_đẽ', 0.5598078966140747),
 ('độc_đáo', 0.5433634519577026),
 ('tuyệt_diệu', 0.5364415049552917),
 ('đặc_biệt', 0.5122414827346802),
 ('quyến_rũ', 0.5009143352508545),
 ('thuần_khiết', 0.4904470443725586),
 ('thông_minh', 0.48917514085769653)]

In [52]:
model.build_vocab(sentences = tiki_sentences, progress_per = 10000, update = True)
model.train(tiki_sentences,  total_examples = model.corpus_count, epochs= 5)

(4864306, 18052655)

In [53]:
vector = model.wv
print(vector)
print(type(vector))

<class 'gensim.models.keyedvectors.Word2VecKeyedVectors'>


In [57]:
vector.most_similar(positive = ["tuyệt_vời"])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('đẹp_đẽ', 0.5426675081253052),
 ('hoàn_hảo', 0.5355318784713745),
 ('hấp_dẫn', 0.5063832998275757),
 ('thuần_khiết', 0.47010737657546997),
 ('tuyệt_diệu', 0.4680679440498352),
 ('đẹp', 0.46634408831596375),
 ('quyến_rũ', 0.46476662158966064),
 ('duyên_dáng', 0.4632873833179474),
 ('độc_đáo', 0.45141980051994324),
 ('sa_đọa', 0.4324997067451477)]

In [58]:
path = "/content/drive/My Drive/DevC/saves/w2vmodel.model"
model.save(path)

In [59]:
path1 = "/content/drive/My Drive/DevC/saves/w2v.wordvectors"
from gensim.models import KeyedVectors
vector.save(path1)

In [60]:
mv = KeyedVectors.load("/content/drive/My Drive/DevC/saves/w2v.wordvectors")
mv["tuyệt"]

array([ 7.4991897e-02, -4.8286668e-03,  2.5539514e-01, -7.3572803e-01,
       -7.0304787e-01, -9.4821341e-02,  1.0701504e+00,  3.8643053e-01,
       -5.3150546e-02, -3.9999002e-01,  1.4225926e+00, -4.1794503e-01,
        6.0349021e-02, -2.1676371e-01, -1.6112508e-01, -2.4176104e-01,
       -2.1886310e-01, -3.6004245e-01,  1.1104667e-01, -3.4502116e-01,
       -4.6274118e-02, -4.5537871e-01, -2.2237659e-01,  2.5177691e+00,
       -1.6608055e-01, -5.5911487e-01,  1.0062743e+00,  1.8106351e+00,
       -8.9139777e-01,  1.0989482e+00, -4.3469581e-01,  3.0336198e-01,
        9.1240859e-01, -9.8775607e-01,  4.3014169e-01, -1.8164005e+00,
        2.7690816e-01, -1.1185282e+00, -4.6997380e-01, -7.4430770e-01,
        9.2464340e-01,  3.9087352e-01, -9.3119085e-01, -7.7045865e-02,
       -1.6981085e-01,  1.3100916e+00, -4.9025401e-01, -7.3231924e-01,
        1.7914956e-02,  6.9733113e-01,  6.0223424e-01,  2.6996449e-01,
        1.9156942e-01, -1.0645190e-01, -7.0118552e-01, -4.1036367e-01,
      

In [61]:
mv.most_similar(positive=["ngon"])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('ngon_lành', 0.6298640966415405),
 ('thơm_ngon', 0.6007251739501953),
 ('rượu_ngon', 0.5335828065872192),
 ('đẫy', 0.5255340337753296),
 ('no_nê', 0.5171617269515991),
 ('bít_tết', 0.49753862619400024),
 ('nấu', 0.4709942042827606),
 ('no_say', 0.46166592836380005),
 ('đói_bụng', 0.45801934599876404),
 ('uống', 0.45713579654693604)]